# ファイルのアップロード方法

## Google Driveにファイルをアップロード

Google Driveに`soturon`フォルダをそのままアップロードしてください。

**フォルダ構造**:
```
Google Drive/
└── MyDrive/
    └── soturon/
        ├── 自己教師あり学習/
        │   └── task/
        │       ├── __init__.py
        │       ├── dataset.py
        │       ├── model.py
        │       └── train.py
        ├── ノイズの付与(共通)/
        │   ├── __init__.py
        │   ├── add_noise.py
        │   ├── power_supply_noise.py
        │   ├── interference_noise.py
        │   └── clock_leakage_noise.py
        ├── eval.py
        └── data_lowF_noise.pickle
```



In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')


# タスク4: マスク予測 + 正則化項

自己教師あり学習によるノイズ検出


## 1. 環境設定


In [ ]:
# このセルは不要（セル1でマウント済み）


In [ ]:
# 必要なライブラリをインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install scikit-learn matplotlib


In [ ]:
# 作業ディレクトリを設定
import os
import sys

# Google Driveにsoturonフォルダをそのままアップロード
project_root = '/content/drive/MyDrive/soturon'  # soturonフォルダのルート
ssl_folder = f'{project_root}/自己教師あり学習'  # 自己教師あり学習フォルダ

if os.path.exists(ssl_folder):
    os.chdir(ssl_folder)
    sys.path.insert(0, ssl_folder)
    sys.path.insert(0, project_root)  # プロジェクトルートも追加
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"⚠️ {ssl_folder} が見つかりません。")
    print("Google Driveにsoturonフォルダを作成して、ファイルをアップロードしてください。")


## 2. データの確認


In [ ]:
# データファイルのパスを設定
pickle_path = '/content/drive/MyDrive/soturon/data_lowF_noise.pickle'

# データが存在するか確認
import os
if os.path.exists(pickle_path):
    print(f"✅ Data file found: {pickle_path}")
else:
    print(f"⚠️ Data file not found at {pickle_path}")
    print("Google Driveのsoturonフォルダにdata_lowF_noise.pickleをアップロードしてください。")


## 3. モデルのインポート


In [ ]:
# ファイルの存在確認
import os
import sys

# 作業ディレクトリを設定（重要！）
# Google Driveにsoturonフォルダをそのままアップロード
project_root = '/content/drive/MyDrive/soturon'  # soturonフォルダのルート
ssl_folder = f'{project_root}/自己教師あり学習'  # 自己教師あり学習フォルダ

if os.path.exists(ssl_folder):
    os.chdir(ssl_folder)
    sys.path.insert(0, ssl_folder)
    sys.path.insert(0, project_root)  # プロジェクトルートも追加（eval.pyやノイズモジュール用）
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"⚠️ {ssl_folder} が見つかりません。")
    print("Google Driveにsoturonフォルダを作成して、ファイルをアップロードしてください。")

# 必要なファイル/フォルダの存在確認
required_files = [
    f'{ssl_folder}/task/__init__.py',
    f'{ssl_folder}/task/dataset.py',
    f'{ssl_folder}/task/model.py',
    f'{ssl_folder}/task/train.py',
    f'{project_root}/ノイズの付与(共通)/__init__.py',
    f'{project_root}/ノイズの付与(共通)/add_noise.py',
    f'{project_root}/ノイズの付与(共通)/power_supply_noise.py',
    f'{project_root}/ノイズの付与(共通)/interference_noise.py',
    f'{project_root}/ノイズの付与(共通)/clock_leakage_noise.py',
    f'{project_root}/eval.py',
    f'{project_root}/data_lowF_noise.pickle'
]

missing_files = []
for file in required_files:
    if not os.path.exists(file):
        # パスを短縮して表示
        if file.startswith(ssl_folder):
            missing_files.append(file.replace(ssl_folder + '/', '自己教師あり学習/'))
        elif file.startswith(project_root):
            missing_files.append(file.replace(project_root + '/', ''))

if missing_files:
    print("⚠️ 以下のファイル/フォルダが見つかりません:")
    for file in missing_files:
        print(f"  - {file}")
    print(f"\n📁 Google Driveの {project_root} にファイルをアップロードしてください。")
    print("   フォルダ構造:")
    print("   soturon/")
    print("   ├── 自己教師あり学習/")
    print("   │   └── task/")
    print("   ├── ノイズの付与(共通)/")
    print("   ├── eval.py")
    print("   └── data_lowF_noise.pickle")
else:
    print("✅ すべての必要なファイルが見つかりました！")

# ノイズモジュールを正しく登録（相対インポートを解決するため）
print("\nnoiseモジュールをsys.modulesに登録中...")
noise_folder_path = os.path.join(project_root, 'ノイズの付与(共通)')

# 既存のnoiseモジュールを削除（キャッシュをクリア）
if 'noise' in sys.modules:
    del sys.modules['noise']
if 'noise.add_noise' in sys.modules:
    del sys.modules['noise.add_noise']

# __pycache__をクリア（古いキャッシュを削除）
import shutil
pycache_path = os.path.join(noise_folder_path, '__pycache__')
if os.path.exists(pycache_path):
    try:
        shutil.rmtree(pycache_path)
        print(f"古いキャッシュを削除: {pycache_path}")
    except Exception as e:
        print(f"キャッシュの削除に失敗（無視します）: {e}")

# __init__.pyの内容を確認し、必要に応じて修正
init_path = os.path.join(noise_folder_path, '__init__.py')
print(f"  __init__.pyのパス: {init_path}")

# Google Colab上のファイルが古い可能性があるため、正しい内容を書き込む
correct_init_content = '''"""
測定系由来のノイズを付与するモジュール
"""

from .power_supply_noise import add_power_supply_noise
from .interference_noise import add_interference_noise
from .clock_leakage_noise import add_clock_leakage_noise
from .add_noise import add_noise_to_interval

__all__ = [
    'add_power_supply_noise',
    'add_interference_noise',
    'add_clock_leakage_noise',
    'add_noise_to_interval'
]
'''

# __init__.pyの内容を確認
try:
    with open(init_path, 'r', encoding='utf-8') as f:
        current_content = f.read()
    
    # 古い内容が含まれている場合は書き換える
    if 'frequency_band_noise' in current_content or 'localized_spike_noise' in current_content:
        print("  古い__init__.pyを検出しました。正しい内容に書き換えます...")
        with open(init_path, 'w', encoding='utf-8') as f:
            f.write(correct_init_content)
        print("  ✓ __init__.pyを更新しました")
    else:
        print("  ✓ __init__.pyは既に正しい内容です")
except Exception as e:
    print(f"  ⚠ __init__.pyの確認に失敗しましたが、続行します: {e}")

# ノイズフォルダをsys.pathに追加（相対インポートを解決するため）
if noise_folder_path not in sys.path:
    sys.path.insert(0, noise_folder_path)

import importlib.util

# noiseパッケージの__init__.pyをロード
init_spec = importlib.util.spec_from_file_location("noise", init_path)
init_module = importlib.util.module_from_spec(init_spec)
sys.modules['noise'] = init_module
# __file__と__path__を設定（相対インポートのため）
init_module.__file__ = init_path
init_module.__path__ = [noise_folder_path]
init_spec.loader.exec_module(init_module)

# add_noise.pyをロード
add_noise_path = os.path.join(noise_folder_path, 'add_noise.py')
add_noise_spec = importlib.util.spec_from_file_location("noise.add_noise", add_noise_path)
add_noise_module = importlib.util.module_from_spec(add_noise_spec)
sys.modules['noise.add_noise'] = add_noise_module
add_noise_module.__file__ = add_noise_path
add_noise_spec.loader.exec_module(add_noise_module)

print("✓ noiseモジュールをsys.modulesに登録しました")
print(f"  利用可能な関数: {init_module.__all__}")

# add_noiseモジュールを直接登録（task/dataset.pyが`from add_noise import`を使うため）
sys.modules['add_noise'] = add_noise_module
print("✓ add_noiseモジュールを直接登録しました")

# タスク4のモジュールをインポート
try:
    from task.dataset import Task4Dataset
    from task.model import Task4BERT
    from task.train import train_task4
    import eval
    print("✅ モジュールのインポートに成功しました！")
except ImportError as e:
    print(f"❌ モジュールのインポートに失敗しました: {e}")
    print("   ファイルが正しくアップロードされているか確認してください。")
    import traceback
    traceback.print_exc()

# モジュールを再読み込み（ファイルを変更した場合）
# ファイルを変更した後は、このセルを再実行してモジュールを再読み込みしてください
import importlib
import sys

modules_to_reload = [
    'task.dataset',
    'task.model',
    'task.train',
    'eval'
]

for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f"✓ {module_name}モジュールを再読み込みしました")

# 再インポート（再読み込み後）
from task.dataset import Task4Dataset
from task.model import Task4BERT
from task.train import train_task4
import eval

print("\n✅ すべてのモジュールを再読み込み完了")


## 4. 学習の実行


In [ ]:
# デバイスの確認
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# チェックポイントを削除して最初から学習する場合（必要に応じて実行）
import os
checkpoint_path = "/content/drive/MyDrive/soturon/自己教師あり学習/task4_output/checkpoint.pth"
if os.path.exists(checkpoint_path):
    print("既存のチェックポイントが見つかりました。削除します...")
    os.remove(checkpoint_path)
    print("✓ チェックポイントを削除しました。最初から学習します。")
else:
    print("チェックポイントは存在しません。最初から学習します。")


In [ ]:
# 学習前にキャッシュを削除してモジュールを再読み込み（古いコードで実行されないようにする）
import os
import sys
import shutil
import importlib
import glob

print("=" * 60)
print("キャッシュを削除してモジュールをリフレッシュ中...")
print("=" * 60)

# 作業ディレクトリを設定
project_root = '/content/drive/MyDrive/soturon'
ssl_folder = f'{project_root}/自己教師あり学習'

if os.path.exists(ssl_folder):
    os.chdir(ssl_folder)
    sys.path.insert(0, ssl_folder)
    sys.path.insert(0, project_root)
    print(f"作業ディレクトリ: {os.getcwd()}")
else:
    print(f"⚠️ {ssl_folder} が見つかりません。")

# 1. __pycache__フォルダを削除
print("\n1. __pycache__フォルダを削除中...")
pycache_dirs = []
for root, dirs, files in os.walk(ssl_folder):
    if '__pycache__' in dirs:
        pycache_path = os.path.join(root, '__pycache__')
        pycache_dirs.append(pycache_path)

for pycache_dir in pycache_dirs:
    try:
        shutil.rmtree(pycache_dir)
        print(f"  ✓ 削除: {pycache_dir.replace(ssl_folder, '自己教師あり学習')}")
    except Exception as e:
        print(f"  ⚠ 削除失敗: {pycache_dir.replace(ssl_folder, '自己教師あり学習')} - {e}")

# 2. .pycファイルを削除
print("\n2. .pycファイルを削除中...")
pyc_files = glob.glob(os.path.join(ssl_folder, '**', '*.pyc'), recursive=True)
for pyc_file in pyc_files:
    try:
        os.remove(pyc_file)
        print(f"  ✓ 削除: {pyc_file.replace(ssl_folder, '自己教師あり学習')}")
    except Exception as e:
        print(f"  ⚠ 削除失敗: {pyc_file.replace(ssl_folder, '自己教師あり学習')} - {e}")

# 3. モジュールをsys.modulesから削除（より徹底的に）
print("\n3. モジュールをsys.modulesから削除中...")
# taskで始まる全てのモジュールを削除
modules_to_remove = [name for name in list(sys.modules.keys()) if name.startswith('task')]
for module_name in modules_to_remove:
    try:
        del sys.modules[module_name]
        print(f"  ✓ 削除: {module_name}")
    except Exception as e:
        print(f"  ⚠ 削除失敗: {module_name} - {e}")

# 4. モジュールを再インポートしてreload
print("\n4. モジュールを再インポートしてreload中...")
try:
    # まずインポート
    import task
    import task.dataset
    import task.model
    import task.train
    
    # 次にreload（これが重要！）
    importlib.reload(task)
    importlib.reload(task.dataset)
    importlib.reload(task.model)
    importlib.reload(task.train)
    
    # 最後に必要な関数/クラスをインポート
    from task.dataset import Task4Dataset
    from task.model import Task4BERT
    from task.train import train_task4
    
    print("  ✓ task.dataset を再読み込み")
    print("  ✓ task.model を再読み込み")
    print("  ✓ task.train を再読み込み")
    
    # 確認: compute_lossとtrain_task4のソースコードを確認（デバッグ用）
    import inspect
    
    # compute_loss関数の確認
    compute_loss_source = inspect.getsource(task.train.compute_loss)
    if '100.0 * noise_penalty' in compute_loss_source:
        print("  ✅ compute_loss: 新しいコード（penalty_weight=100.0）が読み込まれています")
    elif '20.0 * noise_penalty' in compute_loss_source:
        print("  ⚠️ compute_loss: 古いコード（penalty_weight=20.0）が読み込まれています")
    else:
        print("  ⚠️ compute_loss: コードの確認ができませんでした")
    
    # train_task4関数のlambda_regデフォルト値を確認
    train_task4_sig = inspect.signature(task.train.train_task4)
    lambda_reg_default = train_task4_sig.parameters['lambda_reg'].default
    print(f"  train_task4のlambda_regデフォルト値: {lambda_reg_default}")
    
    # ファイルの更新日時を確認（デバッグ用）
    train_py_path = os.path.join(ssl_folder, 'task', 'train.py')
    if os.path.exists(train_py_path):
        import time
        mtime = os.path.getmtime(train_py_path)
        mtime_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(mtime))
        print(f"  train.pyの更新日時: {mtime_str}")
        
        # ファイルの内容を確認（penalty_weightの値を直接確認）
        with open(train_py_path, 'r', encoding='utf-8') as f:
            content = f.read()
            if '100.0 * noise_penalty' in content:
                print("  ✅ train.pyファイルに新しいコード（penalty_weight=100.0）が含まれています")
            elif '20.0 * noise_penalty' in content:
                print("  ⚠️ train.pyファイルに古いコード（penalty_weight=20.0）が含まれています")
            else:
                print("  ⚠️ train.pyファイルの内容を確認できませんでした")
        
except Exception as e:
    print(f"  ⚠ インポートエラー: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 60)
print("✓ キャッシュ削除とモジュール再読み込み完了")
print("=" * 60)
print("\n次のセル（Cell 15）で学習を実行してください。")


In [ ]:
# 学習を実行
# 最初から学習する場合は resume=False に変更してください
# チェックポイントから再開する場合は resume=True のままにしてください

# モジュールの再読み込み（確実に最新のコードを使用するため）
import importlib
import sys

# task.trainモジュールを再読み込み
if 'task.train' in sys.modules:
    importlib.reload(sys.modules['task.train'])

# train_task4関数をインポート（reload後）
from task.train import train_task4

# 確認: lambda_regのデフォルト値を確認（デバッグ用）
import inspect
sig = inspect.signature(train_task4)
lambda_reg_default = sig.parameters['lambda_reg'].default
print(f"確認: train_task4のlambda_regデフォルト値: {lambda_reg_default}")
if lambda_reg_default == 10.0:
    print("  ⚠️ 注意: デフォルト値が10.0ですが、引数で明示的に指定してください")

# 出力ディレクトリを変数に保存（後でグラフ表示で使用）
out_dir = "/content/drive/MyDrive/soturon/自己教師あり学習/task4_output"

model, train_losses, val_losses, best_val_loss, train_recon_losses, val_recon_losses, train_recon_accuracies, val_recon_accuracies = train_task4(
    pickle_path=pickle_path,
    batch_size=4,  # メモリ不足のため8→4に変更
    num_epochs=5,  # エポック数（5エポックで約30分）
    lr=1.5e-3,  # 2e-3 → 1.5e-3に変更（安定した学習）
    val_ratio=0.2,
    device=device,
    out_dir=out_dir,  # 出力先をGoogle Driveに
    resume=False,  # Falseに変更: 最初から学習 / True: チェックポイントから再開
    lambda_reg=1.0,  # 正則化項の重み（1.0に変更、ノイズ区間予測精度向上のため）
    lambda_recon=50.0,  # 復元損失の重み（予測が外れた場合、30.0 → 50.0に増加）
    lambda_recon_correct=100.0,  # 予測が正しい場合の復元損失の重み（予測が外れた場合のlambda_reconより大きく設定）
    lambda_mask=20.0,  # マスク予測損失の重み（新規追加、予測精度を上げるために重要）
    lambda_noise_interval=500.0,  # ノイズ区間予測損失の重み（50.0 → 500.0に変更、ノイズ区間予測精度を大幅に向上）
    lambda_ranking=300.0,  # ランキング損失の重み（30.0 → 300.0に変更、ノイズ区間のアテンション < 正常区間の最小アテンションを強制）
    d_model=64,  # 埋め込み次元（128 → 64に変更、メモリ不足対策）
    n_heads=4,  # アテンションヘッド数（2 → 4に増加）
    num_layers=2,  # Transformerレイヤー数（3 → 2に変更、メモリ不足対策）
    dim_feedforward=128,  # Feedforward層の次元（256 → 128に変更、メモリ不足対策）
    num_intervals=30,
    noise_type='power_supply',  # ノイズタイプ: 'power_supply', 'interference', 'clock_leakage'（use_random_noise=Falseの場合のみ使用）
    use_random_noise=True,  # 3種類のノイズをランダムに使用（推奨）
    noise_level=0.3,  # ノイズレベル（30%）
    margin=0.01,  # 0.2 → 0.01に変更（学習が進みやすくする）
    ranking_margin=0.01,  # ランキング損失のマージン（デフォルト値）
)

print("\n" + "=" * 60)
print("学習完了！学習曲線を表示します")
print("=" * 60)

# 学習が終わったら自動でグラフを表示
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

if len(train_recon_losses) > 0:
    # 1つのグラフを作成（復元損失のみ）
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    # 復元損失（Trainのみ）
    ax.plot(train_recon_losses, color='blue', linewidth=2)
    print(f"✓ Train復元損失データ: {len(train_recon_losses)}エポック")
    print(f"  Train復元損失（MSE）: {train_recon_losses[0]:.6f} → {train_recon_losses[-1]:.6f}")
    
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Reconstruction Loss', fontsize=12)
    ax.set_title('Reconstruction Loss', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # ファイルに保存
    save_path = f"{out_dir}/training_curves_recon.png"
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"\n✓ 学習曲線を '{save_path}' に保存しました")
    
    # 画像を表示
    plt.show()
    print("✓ グラフを表示しました")
else:
    print("⚠️ Train復元損失のデータがありません")


In [ ]:
# 学習曲線を可視化（復元損失のみ）
# ランタイムが切れても実行できるように、pickleファイルから読み込む
import matplotlib.pyplot as plt
import pickle
import os
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# パス設定（Cell 16で設定済みの場合はそのまま使用、なければ再設定）
if 'out_dir' not in globals():
    out_dir = "/content/drive/MyDrive/soturon/自己教師あり学習/task4_output"
else:
    out_dir = globals().get('out_dir', "/content/drive/MyDrive/soturon/自己教師あり学習/task4_output")

# 学習履歴を読み込む（変数が存在しない場合はpickleファイルから読み込む）
if 'train_recon_losses' not in globals():
    # pickleファイルから読み込む
    train_recon_losses_path = os.path.join(out_dir, "train_recon_losses.pkl")
    if os.path.exists(train_recon_losses_path):
        with open(train_recon_losses_path, "rb") as f:
            train_recon_losses = pickle.load(f)
        print(f"✓ pickleファイルから学習履歴を読み込みました: {train_recon_losses_path}")
    else:
        print("⚠️ エラー: 学習履歴の変数もpickleファイルも見つかりません。")
        print(f"   pickleファイルのパス: {train_recon_losses_path}")
        print("   Cell 16で学習を実行してください。")
        train_recon_losses = []

# データの確認と可視化
if len(train_recon_losses) > 0:
    print(f"✓ Train復元損失データ: {len(train_recon_losses)}エポック")
    
    # 1つのグラフを作成（復元損失のみ）
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    # 復元損失（Trainのみ）
    ax.plot(train_recon_losses, color='blue', linewidth=2)
    print(f"  Train復元損失（MSE）: {train_recon_losses[0]:.6f} → {train_recon_losses[-1]:.6f}")
    
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Reconstruction Loss', fontsize=12)
    ax.set_title('Reconstruction Loss', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # ファイルに保存
    save_path = os.path.join(out_dir, "training_curves_recon.png")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"\n✓ 学習曲線を '{save_path}' に保存しました")
    
    # 画像を表示
    plt.show()
    print("✓ グラフを表示しました")
else:
    print("⚠️ Train復元損失のデータがありません")
    print("   Cell 16で学習を実行してください。")


In [ ]:
# ============================================================
# 評価用セル（独立して実行可能）
# 変数が消えても、このセルだけ実行すれば評価できます
# ============================================================

import os
import sys
import importlib
import torch
from torch.utils.data import DataLoader

# パス設定（必要に応じて変更）
project_root = '/content/drive/MyDrive/soturon'
project_path = f'{project_root}/自己教師あり学習'
pickle_path = f'{project_root}/data_lowF_noise.pickle'
best_model_path = f'{project_root}/自己教師あり学習/task4_output/best_val_model.pth'

# 作業ディレクトリを設定
if os.path.exists(project_path):
    os.chdir(project_path)
    sys.path.insert(0, project_path)
    sys.path.insert(0, project_root)
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"⚠️ {project_path} が見つかりません。")
    print("Google Driveにsoturonフォルダを作成して、ファイルをアップロードしてください。")

# モジュールを再読み込み（最新のコードを確実に読み込む）
modules_to_reload = ['task.dataset', 'task.model', 'task.train', 'eval']
for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f"✓ {module_name}モジュールを再読み込みしました")

# インポート
from task.dataset import Task4Dataset
from task.model import Task4BERT
import eval

# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# モデルの初期化と重みの読み込み
seq_len = 3000
model = Task4BERT(
    seq_len=seq_len,
    d_model=64,
    n_heads=2,
    num_layers=2,
    dim_feedforward=128,
    dropout=0.1,
).to(device)

# 学習済みモデルの重みを読み込み
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    print(f"✓ モデルの重みを読み込みました: {best_model_path}")
else:
    print(f"⚠️ モデルの重みファイルが見つかりません: {best_model_path}")
    print("学習を実行してモデルを保存してください。")

print("\n✅ 評価の準備が完了しました")


## 5. 評価の実行


In [ ]:
# テストデータで評価（独立して実行可能）

# パス設定（Cell 18で設定済みの場合はそのまま使用、なければ再設定）
if 'pickle_path' not in globals():
    pickle_path = '/content/drive/MyDrive/soturon/data_lowF_noise.pickle'
if 'device' not in globals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if 'model' not in globals():
    print("⚠️ モデルが初期化されていません。Cell 18を先に実行してください。")

# テストデータセットを作成
test_dataset = Task4Dataset(
    pickle_path=pickle_path,
    num_intervals=30,
    noise_type='power_supply',  # 学習時と同じノイズタイプ（use_random_noise=Falseの場合のみ使用）
    use_random_noise=True,  # 学習時と同じ設定
    noise_level=0.3,  # 学習時と同じノイズレベル
    add_structured_noise_flag=True,  # 全体的な構造化ノイズを付与（学習時と同じ）
)

# テストデータローダーを作成（全データを使用）
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 評価を実行
print("評価を実行中...")
results = eval.evaluate_model(
    model=model,
    dataloader=test_loader,
    method='self_supervised',
    device=device,
    num_intervals=30,
)

# 結果を表示
print("\n評価結果:")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")
print(f"F1-score: {results['f1_score']:.4f}")
if 'loss' in results:
    if results['loss'] == float('inf'):
        print(f"復元損失: 計算不可（ノイズ検知が正しいサンプルがありません）")
    else:
        print(f"復元損失（lossキー）: {results['loss']:.6f} - 最も重要な指標")
if 'reconstruction_loss' in results and results['reconstruction_loss'] is not None:
    print(f"復元損失: {results['reconstruction_loss']:.6f}")
if 'reconstruction_accuracy' in results and results['reconstruction_accuracy'] is not None:
    print(f"復元精度: {results['reconstruction_accuracy']:.2f}%")
if 'roc_auc' in results:
    print(f"ROC-AUC: {results['roc_auc']:.4f}")
if 'attention_diff' in results:
    print(f"アテンションウェイトの差: {results['attention_diff']:.4f}")
    print(f"  (正常区間 - ノイズ区間)")
if 'best_threshold' in results and results['best_threshold'] is not None:
    print(f"最適閾値: {results['best_threshold']:.6f}")
else:
    print(f"予測方法: 最もアテンションが低い区間を予測（閾値なし）")
if 'noise_attention_mean' in results:
    print(f"ノイズ区間の平均アテンション: {results['noise_attention_mean']:.6f}")
if 'normal_attention_mean' in results:
    print(f"正常区間の平均アテンション: {results['normal_attention_mean']:.6f}")


In [ ]:
# 混同行列を可視化
eval.plot_confusion_matrix(
    results['confusion_matrix'],
    title='タスク4: 混同行列',
    save_path="/content/drive/MyDrive/soturon/自己教師あり学習/task4_output/confusion_matrix.png"
)
print("混同行列を保存しました")


In [ ]:
# アテンションウェイトの分布を可視化
if 'attention_weights' in results:
    eval.plot_attention_distribution(
        results['attention_weights'],
        results['labels'],
        num_intervals=30,
        save_path="/content/drive/MyDrive/soturon/自己教師あり学習/task4_output/attention_distribution.png"
    )
    print("アテンションウェイトの分布を保存しました")
